# Data and Train


In [ ]:
import os
import tqdm
import cv2
import random
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Direktori data training
DATADIR = "/content/drive/MyDrive/dataset/KTP & KTM"
training_data = []
width, height = 100, 100

# List of characters to include in the dataset (A-Z and 0-9)
characters = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

# Looping semua image data training untuk diubah menjadi array
for char_name in characters:
    path = os.path.join(DATADIR, char_name)
    class_name = char_name

    for img in tqdm.tqdm(os.listdir(path)):
        try:
            img_array = cv2.imread(os.path.join(DATADIR, char_name, img), cv2.IMREAD_COLOR)

            # Konversi gambar ke grayscale
            gray_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)

            # Tambahkan proses tresholding
            _, threshold_array = cv2.threshold(gray_array, 128, 255, cv2.THRESH_BINARY)

            # Tambahkan proses dilasi
            dilated_array = cv2.dilate(threshold_array, (5, 5), iterations=1)

            # Tambahkan proses erosi
            eroded_array = cv2.erode(dilated_array, (5, 5), iterations=1)

            new_array = cv2.resize(eroded_array, (width, height))
            training_data.append([new_array, class_name])
        except Exception as e:
            pass

random.shuffle(training_data)

X = []
Y = []

for feature, label in training_data:
    X.append(feature)
    Y.append(label)

# Encode string labels to integers
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

# Save the label encoder
np.save('label_encoder.npy', label_encoder.classes_)

# Convert labels to categorical one-hot encoding
Y_one_hot = to_categorical(Y_encoded)

X = np.array(X).reshape(-1, width, height, 1)  # Grayscale images have 1 channel

# Tulis ke file pickle
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("Y.pickle", "wb")
pickle.dump(Y_one_hot, pickle_out)
pickle_out.close()


In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, ZeroPadding2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load file pickle
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)
pickle_in = open("Y.pickle", "rb")
Y = pickle.load(pickle_in)

# Assume Y is already in one-hot encoded format
Y_one_hot = np.array(Y)

X = X / 255.0
width, height = 100, 100

# Input layer
inputs = Input(shape=(width, height, 1))  # Grayscale images have 1 channel

conv_layer = ZeroPadding2D(padding=(2, 2))(inputs)
conv_layer = Conv2D(16, (5, 5), strides=(1, 1), activation='relu')(conv_layer)
conv_layer = MaxPooling2D((2, 2))(conv_layer)
conv_layer = Conv2D(32, (3, 3), strides=(1, 1), activation='relu')(conv_layer)
conv_layer = Conv2D(32, (3, 3), strides=(1, 1), activation='relu')(conv_layer)
conv_layer = MaxPooling2D((2, 2))(conv_layer)
conv_layer = Conv2D(64, (3, 3), strides=(1, 1), activation='relu')(conv_layer)

flaten = Flatten()(conv_layer)
fc_layer = Dense(256, activation='relu')(flaten)
fc_layer = Dense(64, activation='relu')(fc_layer)

# Output layer
num_classes = Y_one_hot.shape[1]
outputs = Dense(num_classes, activation='softmax')(fc_layer)

adam = Adam(lr=0.0001)
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, Y_one_hot, epochs=20, verbose=1)
model.save('anpr_model.h5')


# Predict

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from google.colab.patches import cv2_imshow

# Load the trained model
model = load_model('anpr_model.h5')  # Change the model filename if necessary

# Assuming label_encoder.npy is present in the working directory
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('label_encoder.npy', allow_pickle=True)

# Load a new image for prediction
new_image_path = "/content/test/Screenshot 2023-12-16 at 19.37.33.png"
image = cv2.imread(new_image_path)
img_array = cv2.imread(new_image_path, cv2.IMREAD_GRAYSCALE)

# Apply image processing (thresholding, dilasi, erosi, etc.)
# Contoh: Thresholding
_, thresh = cv2.threshold(img_array, 128, 255, cv2.THRESH_BINARY)

# Contoh: Dilasi
kernel = np.ones((5,5), np.uint8)
dilated = cv2.dilate(thresh, kernel, iterations=1)

# Contoh: Erosi
erosion = cv2.erode(dilated, kernel, iterations=1)

# Resize and normalize the processed image
processed_array = cv2.resize(erosion, (width, height))
processed_array = np.array(processed_array).reshape(-1, width, height, 1)
processed_array = processed_array / 255.0

# Display the processed image
cv2_imshow(processed_array.squeeze() * 255)  # Scaled back to 0-255 for display

# Make prediction
prediction = model.predict(processed_array)

# Decode the prediction back to the original label
predicted_label = label_encoder.inverse_transform(np.argmax(prediction, axis=1))[0]

print("Predicted label: {}".format(predicted_label))


In [ ]:
# Specify the path to your zip file
zip_file_path = '/content/Predik.zip'

# Use the !unzip command to extract the contents
!unzip "$zip_file_path" -d /content/

# Process

In [ ]:
import cv2
import imutils
import numpy as np
import os

# Load gambar
img = cv2.imread('/content/drive/MyDrive/dataset (1)/images/3C_2141720049_5 - Tri Jagad Ariyani.jpg')  # Replace with the actual image filename and path

# Koordinat untuk memotong gambar
coordinates = [(1500, 800), (1500, 1500), (3100, 1500), (3100, 800)]

# Konversi koordinat ke format array numpy
pts = np.array(coordinates, np.int32)
pts = pts.reshape((-1, 1, 2))

# Memotong gambar menggunakan poligon yang ditentukan oleh koordinat
mask = np.zeros_like(img)
cv2.fillPoly(mask, [pts], (255, 255, 255))
result = cv2.bitwise_and(img, mask)

# Mendapatkan koordinat batas hasil crop
x, y, w, h = cv2.boundingRect(np.array(coordinates))

# Crop gambar hasil crop dengan ukuran asli sebelum crop
cropped_result = result[y:y+h, x:x+w]
cropped_result2 = result[y:y+h, x:x+w]

# Menampilkan gambar hasil crop dengan ukuran asli
cv2_imshow(cv2.cvtColor(cropped_result, cv2.COLOR_BGR2RGB))
cv2_imshow(cv2.cvtColor(cropped_result, cv2.COLOR_BGR2GRAY))

# Apply thresholding
ret, bw = cv2.threshold(cv2.cvtColor(cropped_result, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

# Show the thresholded image
cv2_imshow(bw)

# # Define the structuring element for dilation
# kernel = np.ones((3, 3), np.uint8)  # You can adjust the size as needed

# # Apply dilation
# dilation = cv2.dilate(bw, kernel, iterations=2)

# # Show the dilated image
# cv2_imshow(dilation)

# Invert image since we trained our model with a black background
inversion = 255 - bw
# Define the structuring element for dilation
kernel = np.ones((2, 2), np.uint8)  # You can adjust the size as needed

# Apply dilation
dilation = cv2.dilate(inversion, kernel, iterations=2)

# Show the dilated image
cv2_imshow(dilation)

kernel = np.ones((4, 4), np.uint8)  # You can adjust the size as needed

# Apply dilation
erode = cv2.erode(dilation, kernel, iterations=2)

# Check
cv2_imshow(erode)

# Create a directory to save individual character images
save_dir = '/content/character_images/'  # Specify the directory path
os.makedirs(save_dir, exist_ok=True)

# Define function for contour detection
def find_contours(img):
    conts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    conts = imutils.grab_contours(conts)
    conts = sorted(conts, key=cv2.contourArea, reverse=True)

    return conts

conts = find_contours(erode.copy())

# Define margin size
margin = 5  # Adjust according to your requirements

# Counter for saving bounding box images
bbox_count = 0

for c in conts:
    (x, y, w, h) = cv2.boundingRect(c)  # find bounding box based on contour

    # Expand bounding box with margin
    x -= margin
    y -= margin
    w += 2 * margin
    h += 2 * margin

    # Ensure the expanded bounding box is within the image boundaries
    x = max(0, x)
    y = max(0, y)
    w = min(cropped_result.shape[1] - 1 - x, w)
    h = min(cropped_result.shape[0] - 1 - y, h)

    roi = cropped_result[y:y + h, x:x + w]  # get region of interest for char

    # Save individual bounding box image
    bbox_filename = os.path.join(save_dir, f"bbox_{bbox_count}.png")
    cv2.imwrite(bbox_filename, roi)

    # Increment the bounding box count
    bbox_count += 1

# Show bounding box on the original image
cv2.drawContours(cropped_result, conts, -1, (255, 0, 0), 2)
cv2_imshow(cropped_result)


In [ ]:
import cv2
import os
import numpy as np
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from google.colab.patches import cv2_imshow

# Load the trained model
model = load_model('anpr_model.h5')  # Change the model filename if necessary

# Assuming label_encoder.npy is present in the working directory
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('label_encoder.npy', allow_pickle=True)

# Directory containing images for prediction
folder_path = "/content/character_images"  # Change the folder path accordingly

# Process each image in the folder
for img_filename in os.listdir(folder_path):
    # Load an image from the folder
    img_path = os.path.join(folder_path, img_filename)
    img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # Apply image processing (thresholding, dilasi, erosi, etc.)
    # Contoh: Thresholding
    _, thresh = cv2.threshold(img_array, 128, 255, cv2.THRESH_BINARY)

    # Contoh: Dilasi
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(thresh, kernel, iterations=1)

    # Contoh: Erosi
    erosion = cv2.erode(dilated, kernel, iterations=1)

   # Resize and normalize the processed image
    processed_array = cv2.resize(erosion, (100, 100))  # Resize to match the model's input shape
    processed_array = np.array(processed_array).reshape(-1, 100, 100, 1)
    processed_array = processed_array / 255.0

    # Display the processed image (optional)
    cv2_imshow(processed_array.squeeze() * 255)  # Scaled back to 0-255 for display

    # Make prediction
    prediction = model.predict(processed_array)

    # Decode the prediction back to the original label
    predicted_label = label_encoder.inverse_transform(np.argmax(prediction, axis=1))[0]

    print("Predicted label for {}: {}".format(img_filename, predicted_label))


In [ ]:
import cv2
import imutils
import numpy as np
import os
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from google.colab.patches import cv2_imshow

# Load the trained model
model = load_model('anpr_model.h5')  # Change the model filename if necessary

# Assuming label_encoder.npy is present in the working directory
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('label_encoder.npy', allow_pickle=True)

# Create a directory to save individual character images
save_dir = '/content/character_images/'  # Specify the directory path
os.makedirs(save_dir, exist_ok=True)

# Define function for contour detection
def find_contours(img):
    conts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    conts = imutils.grab_contours(conts)
    conts = sorted(conts, key=cv2.contourArea, reverse=True)

    return conts

conts = find_contours(erode.copy())

# Define margin size
margin = 5  # Adjust according to your requirements

# Counter for saving individual character images
char_count = 0

for c in conts:
    (x, y, w, h) = cv2.boundingRect(c)  # find bounding box based on contour

    # Expand bounding box with margin
    x -= margin
    y -= margin
    w += 2 * margin
    h += 2 * margin

    # Ensure the expanded bounding box is within the image boundaries
    x = max(0, x)
    y = max(0, y)
    w = min(cropped_result2.shape[1] - 1 - x, w)
    h = min(cropped_result2.shape[0] - 1 - y, h)

    roi = cropped_result[y:y + h, x:x + w]  # get region of interest for char

    # Save individual character image
    char_filename = os.path.join(save_dir, f"char_{char_count}.png")
    cv2.imwrite(char_filename, roi)

    # Increment the character count
    char_count += 1

    # Convert the character image to grayscale
    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # Resize and normalize the processed image
    processed_array = cv2.resize(roi_gray, (100, 100))  # Resize to match the model's input shape
    processed_array = np.array(processed_array).reshape(-1, 100, 100, 1)  # Convert to 1 channel
    processed_array = processed_array / 255.0

    # Make prediction
    prediction = model.predict(processed_array)

    # Decode the prediction back to the original label
    predicted_label = label_encoder.inverse_transform(np.argmax(prediction, axis=1))[0]

    # Draw bounding box on the original image with predicted label
    cv2.putText(cropped_result, predicted_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    # cv2.rectangle(cropped_result2, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Show bounding box on the original image with predicted labels
cv2_imshow(cropped_result)


# Function


Function


In [ ]:
def Pred_img(img_path) :
    # Load gambar
      img = cv2.imread(img_path)  # Replace with the actual image filename and path

      # Apply thresholding
      ret, bw = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

      # Show the thresholded image
      cv2_imshow(bw)

      # # Define the structuring element for dilation
      # kernel = np.ones((3, 3), np.uint8)  # You can adjust the size as needed

      # # Apply dilation
      # dilation = cv2.dilate(bw, kernel, iterations=2)

      # # Show the dilated image
      # cv2_imshow(dilation)

      # Invert image since we trained our model with a black background
      inversion = 255 - bw
      # Define the structuring element for dilation
      kernel = np.ones((2, 2), np.uint8)  # You can adjust the size as needed

      # Apply dilation
      dilation = cv2.dilate(inversion, kernel, iterations=2)

      # Show the dilated image
      cv2_imshow(dilation)

      kernel = np.ones((4, 4), np.uint8)  # You can adjust the size as needed

      # Apply dilation
      erode = cv2.erode(dilation, kernel, iterations=2)

      # Check
      cv2_imshow(erode)

      # Load the trained model
      model = load_model('anpr_model.h5')  # Change the model filename if necessary

      # Assuming label_encoder.npy is present in the working directory
      label_encoder = LabelEncoder()
      label_encoder.classes_ = np.load('label_encoder.npy', allow_pickle=True)


      # Define function for contour detection
      def find_contours(img):
          conts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
          conts = imutils.grab_contours(conts)
          conts = sorted(conts, key=cv2.contourArea, reverse=True)

          return conts

      conts = find_contours(erode.copy())

      # Define margin size
      margin = 5  # Adjust according to your requirements

      # Counter for saving individual character images
      char_count = 0

      for c in conts:
          (x, y, w, h) = cv2.boundingRect(c)  # find bounding box based on contour

          # Expand bounding box with margin
          x -= margin
          y -= margin
          w += 2 * margin
          h += 2 * margin

          # Ensure the expanded bounding box is within the image boundaries
          x = max(0, x)
          y = max(0, y)
          w = min(img.shape[1] - 1 - x, w)
          h = min(img.shape[0] - 1 - y, h)

          roi = img[y:y + h, x:x + w]  # get region of interest for char

          # Increment the character count
          char_count += 1

          # Check if roi is not empty
          if not roi.size == 0:
              # Convert the character image to grayscale
              roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

              # Resize and normalize the processed image
              processed_array = cv2.resize(roi_gray, (100, 100))  # Resize to match the model's input shape
              processed_array = np.array(processed_array).reshape(-1, 100, 100, 1)  # Convert to 1 channel
              processed_array = processed_array / 255.0

              # Make prediction
              prediction = model.predict(processed_array)

              # Decode the prediction back to the original label
              predicted_label = label_encoder.inverse_transform(np.argmax(prediction, axis=1))[0]

              # Draw bounding box on the original image with predicted label
              cv2.putText(img, predicted_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
              cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)  # Draw bounding box

      # Show bounding box on the original image with predicted labels
      cv2_imshow(img)
      return img


In [ ]:
image1 = Pred_img("/content/drive/MyDrive/dataset (1)/images/3C_2141720049_5 - Tri Jagad Ariyani.jpg")

Function2

# Version 2

In [ ]:
import cv2
import imutils
import numpy as np
import os
from imutils.contours import sort_contours
from google.colab.patches import cv2_imshow
# Load gambar
img = cv2.imread('/content/drive/MyDrive/dataset (1)/images/3C_2141720049_5 - Tri Jagad Ariyani.jpg')  # Replace with the actual image filename and path

# Koordinat untuk memotong gambar
coordinates = [(1500, 800), (1500, 1500), (3100, 1500), (3100, 800)]

# Konversi koordinat ke format array numpy
pts = np.array(coordinates, np.int32)
pts = pts.reshape((-1, 1, 2))

# Memotong gambar menggunakan poligon yang ditentukan oleh koordinat
mask = np.zeros_like(img)
cv2.fillPoly(mask, [pts], (255, 255, 255))
result = cv2.bitwise_and(img, mask)

# Mendapatkan koordinat batas hasil crop
x, y, w, h = cv2.boundingRect(np.array(coordinates))

# Crop gambar hasil crop dengan ukuran asli sebelum crop
cropped_result = result[y:y+h, x:x+w]

# Menampilkan gambar hasil crop dengan ukuran asli
cv2_imshow(cv2.cvtColor(cropped_result, cv2.COLOR_BGR2RGB))
cv2_imshow(cv2.cvtColor(cropped_result, cv2.COLOR_BGR2GRAY))

gray = cv2.cvtColor(cropped_result, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)
dilated = cv2.dilate(blur, np.ones((3,3)))
adaptive = cv2.adaptiveThreshold(dilated, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 1)
invertion = 255 - adaptive
erode = cv2.erode(invertion, np.ones((3,3)))
dilated = cv2.dilate(erode, np.ones((2,2)))

# Check
cv2_imshow(dilated)

# Define function for contour detection
def find_contours(img):
  conts = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  conts = imutils.grab_contours(conts)
  conts = sort_contours(conts, method='left-to-right')[0]

  return conts

In [ ]:
conts = find_contours(dilated.copy())

In [ ]:
# Get the char based on contour with margin in bounding box

# Setup min/max width/height for char
min_w, max_w = 30, 160
min_h, max_h = 34, 140
margin = 5  # Set the margin value

img_copy = cropped_result.copy()  # original image for plotting contour result
filtered_conts = []

# Create a directory to save individual character images
save_dir = '/content/character_images2/'
os.makedirs(save_dir, exist_ok=True)

for c in conts:
    (x, y, w, h) = cv2.boundingRect(c)  # find bounding box based on contour

    # Expand bounding box with margin
    x -= margin
    y -= margin
    w += 2 * margin
    h += 2 * margin

    # Ensure the expanded bounding box is within the image boundaries
    x = max(0, x)
    y = max(0, y)
    w = min(gray.shape[1] - 1 - x, w)
    h = min(gray.shape[0] - 1 - y, h)

    if (w >= min_w and w <= max_w) and (h >= min_h and h <= max_h):  # if pixel follows this rule, it is considered as a char
        filtered_conts.append(c)
        roi = gray[y:y+h, x:x+w]  # get region of interest for char
        thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        cv2_imshow(thresh)  # check

        # Save individual character image
        char_filename = os.path.join(save_dir, f"char_{len(filtered_conts)}.png")
        cv2.imwrite(char_filename, roi)  # Save the character image instead of the bounding box

        # Build bounding box on the original image (for visualization purposes)
        cv2.rectangle(img_copy, (x, y), (x+w, y+h), (255, 0, 0), 2)


In [ ]:
# Define the functions
def extract_roi(img, margin=2):
    roi = img[margin:-margin, margin:-margin]
    return roi

def thresholding(img):
    thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    return thresh

def resize_img(img):
    if img is None:
        # Handle the case where thresholding did not produce a valid result
        return None

    resized = cv2.resize(img, (100, 100))  # Sesuaikan dengan dimensi yang diharapkan oleh model
    resized = np.expand_dims(resized, axis=-1)
    resized = np.expand_dims(resized, axis=0)

    return resized

In [ ]:
# Demo for enlarge
(x, y, w, h) = cv2.boundingRect(filtered_conts[2])
test_image = thresholding(gray[y:y+h, x:x+w])

# show original test image
cv2_imshow(test_image)

# Show enlarge test image
cv2_imshow(cv2.resize(test_image, (28,28)))

In [ ]:
def normalization(img):
    img = img.astype('float32') / 255.0  # Convert to floating point and scale to [0, 1]
    img = np.expand_dims(img, axis=-1)  # Add depth

    # Perform any additional normalization or preprocessing steps if needed

    return img


In [ ]:
# Create a directory to save processed character images
processed_dir = '/content/processed_character_images/'
os.makedirs(processed_dir, exist_ok=True)

char_dir = '/content/character_images2'

# Process each character image in the input directory
for char_filename in os.listdir(char_dir):
    char_path = os.path.join(char_dir, char_filename)
    char_img = cv2.imread(char_path, cv2.IMREAD_GRAYSCALE)

    # Extract the region of interest
    roi = extract_roi(char_img)

    # Apply thresholding
    thresh = thresholding(roi)

    # Save the processed character image
    output_filename = os.path.join(processed_dir, f"processed_{char_filename}")
    cv2.imwrite(output_filename, thresh)


In [ ]:
model = load_model('/content/anpr_model.h5')

In [ ]:
digits = '0123456789'
letters = 'ABCDEFGHIJKLMNOPQRSTUVWZYZabcdefghijklmnopqrstuvwxyz'
char_list = digits + letters
char_list = [ch for ch in char_list]

print(char_list)

In [ ]:
import cv2
import os
import numpy as np
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from google.colab.patches import cv2_imshow

# Load the trained model
model = load_model('anpr_model.h5')  # Change the model filename if necessary

# Assuming label_encoder.npy is present in the working directory
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('label_encoder.npy', allow_pickle=True)

# Directory containing images for prediction
folder_path = "/content/character_images2"  # Change the folder path accordingly

# Process each image in the folder
for img_filename in os.listdir(folder_path):
    # Load an image from the folder
    img_path = os.path.join(folder_path, img_filename)
    img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # Apply image processing (thresholding, dilasi, erosi, etc.)
    # Contoh: Thresholding
    _, thresh = cv2.threshold(img_array, 128, 255, cv2.THRESH_BINARY)

    # Contoh: Dilasi
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(thresh, kernel, iterations=1)

    # Contoh: Erosi
    erosion = cv2.erode(dilated, kernel, iterations=1)

   # Resize and normalize the processed image
    processed_array = cv2.resize(erosion, (100, 100))  # Resize to match the model's input shape
    processed_array = np.array(processed_array).reshape(-1, 100, 100, 1)
    processed_array = processed_array / 255.0

    # Display the processed image (optional)
    cv2_imshow(processed_array.squeeze() * 255)  # Scaled back to 0-255 for display

    # Make prediction
    prediction = model.predict(processed_array)

    # Decode the prediction back to the original label
    predicted_label = label_encoder.inverse_transform(np.argmax(prediction, axis=1))[0]

    print("Predicted label for {}: {}".format(img_filename, predicted_label))


In [ ]:
# Display the original image
cv2_imshow(img_copy)

# Process each image in the folder
for img_filename in os.listdir(folder_path):
    # Load an image from the folder
    img_path = os.path.join(folder_path, img_filename)
    img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    # Apply image processing (thresholding, dilasi, erosi, etc.)
    # Contoh: Thresholding
    _, thresh = cv2.threshold(img_array, 128, 255, cv2.THRESH_BINARY)

    # Contoh: Dilasi
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(thresh, kernel, iterations=1)

    # Contoh: Erosi
    erosion = cv2.erode(dilated, kernel, iterations=1)

    # Resize and normalize the processed image
    processed_array = cv2.resize(erosion, (100, 100))  # Resize to match the model's input shape
    processed_array = np.array(processed_array).reshape(-1, 100, 100, 1)
    processed_array = processed_array / 255.0

    # Make prediction
    prediction = model.predict(processed_array)

    # Decode the prediction back to the original label
    predicted_label = label_encoder.inverse_transform(np.argmax(prediction, axis=1))[0]

    # Ensure count is within the range of filtered_conts
    count = int(img_filename.split('_')[1].split('.')[0])
    if count < len(filtered_conts):
        # Get the bounding box coordinates from filtered_conts
        x, y, w, h = cv2.boundingRect(filtered_conts[count])

        # Draw bounding box and label on the original image
        color = (0, 255, 0)  # Green color for bounding box
        thickness = 2

        # Draw bounding box
        cv2.rectangle(img_copy, (x, y), (x+w, y+h), color, thickness)

        # Draw label
        label_position = (x, y - 10)  # Above the bounding box
        cv2.putText(img_copy, predicted_label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, thickness)

# Display the result
cv2_imshow(img_copy)
